# Proyecto: Análisis de datos de ventas en un restaurante


Algunas funciones aplicadas en esta práctica son:

`.info()` <- Extrae la información de la estructura del dataframe

`.describe()` <- Extrae la información estadística del dataframe

`.value_counts()` <- Realiza un conteo de registros que pertenecen a cada clase de valores en una columna

`.groupby()` <- Agrupa información del dataframe 

`.apply()` <- Aplica funciones definidas por el usuario a un dataframe agrupado

`.loc[]` <- Extrae subconjuntos de información

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
pd.options.display.float_format = '{:,.3f}'.format

In [3]:
# Extraemos una base de datos sobre propinas 'tips' en un restaurante
df = sns.load_dataset('tips')
df

,total_bill,tip,sex,smoker,day,time,size
0,16.990,1.010,Female,No,Sun,Dinner,2
1,10.340,1.660,Male,No,Sun,Dinner,3
2,21.010,3.500,Male,No,Sun,Dinner,3
3,23.680,3.310,Male,No,Sun,Dinner,2
4,24.590,3.610,Female,No,Sun,Dinner,4
...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3
240,27.180,2.000,Female,Yes,Sat,Dinner,2
241,22.670,2.000,Male,Yes,Sat,Dinner,2
242,17.820,1.750,Male,No,Sat,Dinner,2


In [4]:
# Obtenemos información estructural del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [5]:
# Obtenemos información estadística del dataframe
df.describe()

,total_bill,tip,size
count,244.000,244.000,244.000
mean,19.786,2.998,2.570
std,8.902,1.384,0.951
min,3.070,1.000,1.000
25%,13.348,2.000,2.000
50%,17.795,2.900,2.000
75%,24.127,3.562,3.000
max,50.810,10.000,6.000


In [7]:
# Extraemos el tercer cuartil de los datos:
df.describe().loc[ ['75%'] ]

,total_bill,tip,size
75%,24.127,3.562,3.000


In [9]:
# Numero de clases en los elemtos de la columna ['day']
df['day'].nunique()

4

In [12]:
# Obtenemos el conteo de registros por día
df['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [13]:
# Obtenemos los registros como porcentajes con respecto al total de visitas:
( df['day'].value_counts() / df['day'].value_counts().sum() )*100

Sat    35.656
Sun    31.148
Thur   25.410
Fri     7.787
Name: day, dtype: float64

In [14]:
# Agregamos una columna que incluya el porcentaje de la propina con respecto al pago de la cuenta:
df['prct_tip'] = df['tip']/df['total_bill']
df

,total_bill,tip,sex,smoker,day,time,size,prct_tip
0,16.990,1.010,Female,No,Sun,Dinner,2,0.059
1,10.340,1.660,Male,No,Sun,Dinner,3,0.161
2,21.010,3.500,Male,No,Sun,Dinner,3,0.167
3,23.680,3.310,Male,No,Sun,Dinner,2,0.140
4,24.590,3.610,Female,No,Sun,Dinner,4,0.147
...,...,...,...,...,...,...,...,...
239,29.030,5.920,Male,No,Sat,Dinner,3,0.204
240,27.180,2.000,Female,Yes,Sat,Dinner,2,0.074
241,22.670,2.000,Male,Yes,Sat,Dinner,2,0.088
242,17.820,1.750,Male,No,Sat,Dinner,2,0.098


In [15]:
# Obtenemos el promedio de la información agupada de acuerdo al género:
df.groupby(['sex']).mean()

,total_bill,tip,size,prct_tip
sex,,,,
Male,20.744,3.090,2.631,0.158
Female,18.057,2.833,2.460,0.166


In [16]:
# Obtenemos información estadística de la variable ['prct_tip'], agrupada por ['sex']
df.groupby(['sex'])[ ['prct_tip'] ].describe()

prct_tip                                          
          count  mean   std   min   25%   50%   75%   max
sex                                                      
Male    157.000 0.158 0.065 0.036 0.121 0.153 0.186 0.710
Female   87.000 0.166 0.054 0.056 0.140 0.156 0.194 0.417

In [17]:
# Obtenemos información estadística de las variables ['total_bill','prct_tip'], agrupada por ['sex']
df.groupby(['sex'])[ ['total_bill','prct_tip'] ].describe()

total_bill                                                prct_tip  \
            count   mean   std   min    25%    50%    75%    max    count   
sex                                                                         
Male      157.000 20.744 9.246 7.250 14.000 18.350 24.710 50.810  157.000   
Female     87.000 18.057 8.009 3.070 12.750 16.400 21.520 44.300   87.000   

                                                  
        mean   std   min   25%   50%   75%   max  
sex                                               
Male   0.158 0.065 0.036 0.121 0.153 0.186 0.710  
Female 0.166 0.054 0.056 0.140 0.156 0.194 0.417

In [18]:
# Definimos una función que transforma euros a dolares: 
def mean_eur2dollars(x):
    return np.mean(x)*1.12

In [19]:
# Aplicamos la función (definida anteriormente) a los datos agrupados 
df.groupby(['sex','time'])[ ['total_bill','prct_tip'] ].apply(mean_eur2dollars)

total_bill  prct_tip
sex    time                        
Male   Lunch       20.214     0.186
       Dinner      24.037     0.174
Female Lunch       18.300     0.182
       Dinner      21.519     0.190

In [20]:
# Aplicamos una función de Numpy a los datos agrupados 
S = df.groupby(['sex','time'])[ ['total_bill','prct_tip'] ].apply(np.std)
S

total_bill  prct_tip
sex    time                        
Male   Lunch        7.832     0.045
       Dinner       9.423     0.069
Female Lunch        7.393     0.034
       Dinner       8.123     0.063

## Extracción de información con .loc[]

In [21]:
S.loc[['Female'],:]

total_bill  prct_tip
sex    time                        
Female Lunch        7.393     0.034
       Dinner       8.123     0.063

In [22]:
S.loc[['Female'],['Dinner'],:]

,,total_bill,prct_tip
sex,time,,
Female,Dinner,8.123,0.063


In [23]:
S.loc[['Female'],['Dinner'],:]['prct_tip']

sex     time  
Female  Dinner   0.063
Name: prct_tip, dtype: float64

In [24]:
S.loc[['Female'],['Dinner'],:][['prct_tip']]

,,prct_tip
sex,time,
Female,Dinner,0.063
